In [1]:
import pandas as pd

# Load questions
df = pd.read_excel("qa_results.xlsx")
df

,Question,Unnamed: 1,Expected Answer,Model Answer,Cypher Answer,NL Answer
0,What is the subject code for Predictive Analyt...,NaN,The subject code for Predictive Analytics is C...,"MATCH (s:Subject {name: ""Predictive Analytics""...",[{'s.code': 'COMP7023'}],The subject code for Predictive Analytics is C...
1,What is the subject name of MATH 7016?,NaN,The subject name is The Nature of Data.,"MATCH (s:Subject {code: ""MATH7016""})\nRETURN s...","[{'s.code': 'MATH7016', 's.name': 'The Nature ...",The subject name of MATH 7016 is The Nature of...
2,Which subjects list MATH7016 as a prerequisite?\n,NaN,"MATH7016 is a prerequisite for COMP7016, MATH7...",MATCH (subject:Subject)-[:REQUIRES]->(preqsub:...,"[{'subject.code': 'COMP7016', 'subject.name': ...",MATH7016 is a prerequisite for Visualisation (...
3,Does MATH 7017 have any pre-requisites?,NaN,"Yes, MATH 7017 has prerequisites of MATH 7016","MATCH (s:Subject {code: ""MATH7017""})-[:REQUIRE...",[{'has_prerequisites': True}],"Yes, MATH 7017 has prerequisites."
4,Which subjects are offered in Spring 2025? (Li...,NaN,"COMP 7024, COMP 7006, MATH 7016, MATH 7002, CO...",MATCH (s:Subject)-[:OFFERED_IN]->(tp:TeachingP...,"[{'s.code': 'COMP7006'}, {'s.code': 'MATH7002'...","COMP7006, MATH7002, COMP7023, INFO7016, INFO70..."
5,Which subject are taught exclusively in Autumn...,"Which subjects are only available in Autumn, n...",COMP7003 and MATH7017 are taught exclusively i...,MATCH (s:Subject)-[:OFFERED_IN]->(tp:TeachingP...,"[{'s.code': 'COMP7003', 's.name': 'Big Data'},...",Big Data (COMP7003) and Probabilistic Graphica...
6,List all subject codes that are taught in both...,Which subjects are offered in both Spring and ...,"COMP 7024, MATH 7016, INFO 7016, INFO 7017 are...",MATCH (s:Subject)-[:OFFERED_IN]->(tpSpring:Tea...,"[{'s.code': 'INFO7016', 's.name': 'Postgraduat...",The following subject codes are taught in both...
7,List ALL teaching periods in one line of MATH ...,In which semesters is MATH7017 taught? (List a...,Autumn 2025,"MATCH (s:Subject {code: ""MATH7017""})-[:OFFERED...",[{'tp.name': 'Autumn 2025'}],Autumn 2025.
8,Which subjects belong to the Statistics discip...,NaN,Social Media Intelligence,MATCH (s:Subject)-[:HAS_OUTCOME]->(o:Outcome)\...,"[{'s.code': 'COMP7025', 's.name': 'Social Medi...",Social Media Intelligence.
9,Which subjects have a credit point restriction?,NaN,INFO 7016 has a credit point restriction.,MATCH (s:Subject)-[:RESTRICTED_BY]->(r:Restric...,"[{'s.code': 'INFO7016', 's.name': 'Postgraduat...",INFO7016 (Postgraduate Project A) has a credit...


In [2]:
from langchain_community.llms import Ollama
import ast

# Initialize LLM
llm = Ollama(model="gemma3:12b", temperature=0)

# New column to hold natural language answers
nl_answers = []

for idx, row in df.iterrows():
    question = row["Question"]
    cypher_answer = row["Cypher Answer"]

    # Try to parse Cypher Answer string into a Python object
    try:
        parsed = ast.literal_eval(cypher_answer) if isinstance(cypher_answer, str) else cypher_answer
    except Exception:
        parsed = cypher_answer  # fallback if parsing fails

    prompt = f"""
Question: {question}
Cypher Answer: {parsed}

Ignore the subject name and convert the Cypher Answer into a short and direct natural answer that answers the question clearly.
    """

    # Get LLM output
    try:
        response = llm.invoke(prompt)
    except Exception as e:
        response = f"[LLM Error: {e}]"

    print(f"Row {idx+1} done ✅")
    nl_answers.append(response)

C:\Users\Admin\AppData\Local\Temp\ipykernel_8036\4098518940.py:5: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:12b", temperature=0)


Row 1 done ✅
Row 2 done ✅
Row 3 done ✅
Row 4 done ✅
Row 5 done ✅
Row 6 done ✅
Row 7 done ✅
Row 8 done ✅
Row 9 done ✅
Row 10 done ✅
Row 11 done ✅
Row 12 done ✅
Row 13 done ✅
Row 14 done ✅
Row 15 done ✅
Row 16 done ✅
Row 17 done ✅
Row 18 done ✅


In [3]:
nl_answers

['COMP7023.',
 'MATH 7016 is called "The Nature of Data".',
 'COMP7016, MATH7002, and MATH7017 list MATH7016 as a prerequisite.',
 'Yes, it does.',
 'COMP7006, MATH7002, COMP7023, INFO7016, INFO7001, INFO7017, COMP7024, MATH7016.',
 'COMP7003 and MATH7017.',
 'INFO7016, INFO7017, COMP7024, MATH7016.\n',
 'Autumn 2025',
 'COMP7025.',
 'INFO7016 has a credit point restriction.',
 'COMP7025.',
 'COMP7003, COMP7006, COMP7023, INFO7016, COMP7025, COMP7024, and MATH7016 have no prerequisites.',
 'COMP7006 and INFO7001.',
 'Network models are covered in networking.\n',
 'Upon completion of COMP 7024, students will be able to: manage data with Excel, extract, transform, and load data using R and R-Studio, create complex R programs for data science, access databases with SQL, apply simulation techniques, and create reports using Markdown and R-Markdown.',
 'Students must be enrolled in a postgraduate program and have completed 60 credit points.',
 '8\n',
 'COMP7003 is worth 10 credit points.\n'

In [4]:
# Add new column to dataframe
df["NL Answer"] = nl_answers

# Save to new Excel file
df.to_excel("qa_results.xlsx", index=False)
print("Saved")

Saved


In [12]:
import pandas as pd

# Load questions
df2 = pd.read_excel("qa2_results.xlsx")
df2

,Question,Expected Answer,Cypher Answer
0,I’m enrolling in Spring and haven’t completed ...,"MATH 7016, COMP 7023, COMP 7024, COMP 7006","[['COMP7003'], ['COMP7025']]"
1,"I’ve completed MATH7016, COMP7024, COMP7003, a...","COMP 7006, INFO7001, and COMP 7023","[['COMP7006', 'Data Science'], ['COMP7023', 'P..."
2,"After finishing MATH7002, COMP7006, and COMP70...","MATH 7016, COMP 7025, COMP 7003, COMP 7024","[['COMP7003', 'Big Data'], ['COMP7016', 'Visua..."
3,"I’ve already completed MATH 7002, COMP 7006, a...","COMP 7024, INFO 7016","[['INFO7016', 'Postgraduate Project A'], ['COM..."
4,I want to learn about machine learning. Which ...,"COMP 7023, INFO 7001, COMP 7006, MATH 7017","[['COMP7006', 'Data Science'], ['COMP7023', 'P..."
5,Which subjects are project-based or include ma...,INFO 7017,"[['INFO7017', 'Postgraduate Project B', 'Postg..."
6,Which subjects introduce Python programming?,COMP 7023,"[['COMP7023', 'Predictive Analytics']]"
7,Which subjects help develop programming skills?,"COMP 7024, COMP 7023, INFO 7001","[['COMP7023', 'Predictive Analytics'], ['COMP7..."
8,Are there subjects with multi-level prerequisi...,MATH 7002,NaN
9,I failed Predictive Analytics in Spring 2024. ...,Spring 2025,NaN


In [13]:
from langchain_community.llms import Ollama
import ast

# Initialize LLM
llm = Ollama(model="gemma3:12b", temperature=0)

# New column to hold natural language answers
nl_answers2 = []

for idx, row in df2.iterrows():
    question = row["Question"]
    cypher_answer = row["Cypher Answer"]

    # Try to parse Cypher Answer string into a Python object
    try:
        parsed = ast.literal_eval(cypher_answer) if isinstance(cypher_answer, str) else cypher_answer
    except Exception:
        parsed = cypher_answer  # fallback if parsing fails

    prompt = f"""
Question: {question}
Cypher Answer: {parsed}

Convert the Cypher Answer into a short and direct natural answer that answers the question clearly.
    """

    # Get LLM output
    try:
        response = llm.invoke(prompt)
    except Exception as e:
        response = f"[LLM Error: {e}]"

    print(f"Row {idx+1} done ✅")
    nl_answers2.append(response)

Row 1 done ✅
Row 2 done ✅
Row 3 done ✅
Row 4 done ✅
Row 5 done ✅
Row 6 done ✅
Row 7 done ✅
Row 8 done ✅
Row 9 done ✅
Row 10 done ✅
Row 11 done ✅
Row 12 done ✅
Row 13 done ✅
Row 14 done ✅
Row 15 done ✅
Row 16 done ✅
Row 17 done ✅
Row 18 done ✅


In [14]:
nl_answers2

['You can take COMP7003 and COMP7025 in the Spring semester without any prerequisites.',
 'Based on your completed courses, you can take COMP7006, COMP7023, INFO7016, or INFO7001 in Spring.',
 'Based on completing MATH7002, COMP7006, and COMP7023, you can take COMP7003, COMP7016, INFO7016, COMP7025, INFO7017, MATH7017, COMP7024, or MATH7016 in the next Autumn semester.',
 'Based on your completed courses, you can take INFO7016 or COMP7024 next Spring.',
 'To learn machine learning, focus on these subjects: Data Science, Predictive Analytics, Advanced Machine Learning, and Probabilistic Graphical Models.',
 'INFO7017 includes a major project component.\n',
 'Predictive Analytics (COMP7023) introduces Python programming.\n',
 'Subjects like Predictive Analytics (COMP7023) and Programming for Data Science (COMP7024) help develop programming skills.',
 'Yes, subjects can have multiple levels of prerequisites.',
 'The information needed to answer that question is not available.',
 'The foll

In [15]:
# Add new column to dataframe
df2["NL Answer"] = nl_answers2

# Save to new Excel file
df2.to_excel("qa2_results.xlsx", index=False)